<a href="https://colab.research.google.com/github/kakimovaazhar-coder/alash_website/blob/main/mod1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q ultralytics
!pip install -q supervision
import os
from IPython.display import clear_output
clear_output()

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

SAVE_DIR = "/content/drive/MyDrive/Helmet_YOLOv8_results"
os.makedirs(SAVE_DIR, exist_ok=True)

Mounted at /content/drive


In [ ]:
import os

!mkdir -p ~/.kaggle

!mv kaggle.json ~/.kaggle/kaggle.json

!chmod 600 ~/.kaggle/kaggle.json

!mkdir -p /content/datasets

print("Kaggle API setup and dataset directory created.")

mv: cannot stat 'kaggle.json': No such file or directory
Kaggle API setup and dataset directory created.


In [ ]:
import os

!kaggle datasets download -d andrewmvd/hard-hat-detection -p /content/datasets

print("Dataset download initiated.")

Dataset URL: https://www.kaggle.com/datasets/andrewmvd/hard-hat-detection
License(s): CC0-1.0
 99% 1.21G/1.22G [00:08<00:00, 237MB/s]
100% 1.22G/1.22G [00:08<00:00, 154MB/s]
Dataset download initiated.


In [ ]:
import os

!unzip -q /content/datasets/hard-hat-detection.zip -d /content/datasets

!rm /content/datasets/hard-hat-detection.zip

print("Dataset unpacked and zip file removed.")

replace /content/datasets/annotations/hard_hat_workers0.xml? [y]es, [n]o, [A]ll, [N]one, [r]ename: Dataset unpacked and zip file removed.


In [ ]:
import numpy as np
import pandas as pd
from pathlib import Path
from xml.dom.minidom import parse
from shutil import copyfile
import os

In [ ]:
!mkdir -p Dataset/labels
!mkdir -p Dataset/images

In [ ]:
classes = ['helmet','head','person']

In [ ]:
def convert_annot(size , box):
    x1 = int(box[0])
    y1 = int(box[1])
    x2 = int(box[2])
    y2 = int(box[3])

    dw = np.float32(1. / int(size[0]))
    dh = np.float32(1. / int(size[1]))

    w = x2 - x1
    h = y2 - y1
    x = x1 + (w / 2)
    y = y1 + (h / 2)

    x = x * dw
    w = w * dw
    y = y * dh
    h = h * dh
    return [x, y, w, h]

In [ ]:
def save_txt_file(img_jpg_file_name, size, img_box):
    save_file_name = '/kaggle/working/Dataset/labels/' +  img_jpg_file_name + '.txt'

    with open(save_file_name ,'a+') as file_path:
        for box in img_box:

            cls_num = classes.index(box[0])

            new_box = convert_annot(size, box[1:])

            file_path.write(f"{cls_num} {new_box[0]} {new_box[1]} {new_box[2]} {new_box[3]}\n")

        file_path.flush()
        file_path.close()

In [ ]:
def get_xml_data(file_path, img_xml_file):
    img_path = file_path + '/' + img_xml_file + '.xml'

    dom = parse(img_path)
    root = dom.documentElement
    img_name = root.getElementsByTagName("filename")[0].childNodes[0].data
    img_size = root.getElementsByTagName("size")[0]
    objects = root.getElementsByTagName("object")
    img_w = img_size.getElementsByTagName("width")[0].childNodes[0].data
    img_h = img_size.getElementsByTagName("height")[0].childNodes[0].data
    img_c = img_size.getElementsByTagName("depth")[0].childNodes[0].data

    img_box = []
    for box in objects:
        cls_name = box.getElementsByTagName("name")[0].childNodes[0].data
        x1 = int(box.getElementsByTagName("xmin")[0].childNodes[0].data)
        y1 = int(box.getElementsByTagName("ymin")[0].childNodes[0].data)
        x2 = int(box.getElementsByTagName("xmax")[0].childNodes[0].data)
        y2 = int(box.getElementsByTagName("ymax")[0].childNodes[0].data)

        img_jpg_file_name = img_xml_file + '.jpg'
        img_box.append([cls_name, x1, y1, x2, y2])


    save_txt_file(img_xml_file, [img_w, img_h], img_box)

In [ ]:
import os

annotations_dir = '/content/datasets/hard-hat-detection/annotations'

os.makedirs(LABELS_DIR, exist_ok=True)

def save_txt_file(img_jpg_file_name, size, img_box):
    save_file_name = os.path.join(LABELS_DIR, img_jpg_file_name + '.txt')

    with open(save_file_name ,'a+') as file_path:
        for box in img_box:

            cls_num = classes.index(box[0])

            new_box = convert_annot(size, box[1:])

            file_path.write(f"{cls_num} {new_box[0]} {new_box[1]} {new_box[2]} {new_box[3]}\n")

        file_path.flush()
        file_path.close()


In [ ]:
import os

annotations_dir = '/content/datasets/annotations'

LABELS_DIR = '/content/datasets/labels'

os.makedirs(LABELS_DIR, exist_ok=True)

def save_txt_file(img_jpg_file_name, size, img_box):
    save_file_name = os.path.join(LABELS_DIR, img_jpg_file_name + '.txt')

    with open(save_file_name ,'a+') as file_path:
        for box in img_box:

            cls_num = classes.index(box[0])

            new_box = convert_annot(size, box[1:])

            file_path.write(f"{cls_num} {new_box[0]} {new_box[1]} {new_box[2]} {new_box[3]}\n")

        file_path.flush()
        file_path.close()

In [ ]:
files = os.listdir(annotations_dir)
for file in files:
    file_xml = file.split(".")
    if file_xml[-1] == 'xml':
        get_xml_data(annotations_dir, file_xml[0])
print(f"Processed {len(files)} XML annotation files and converted them to YOLO format labels in {LABELS_DIR}")

Processed 5000 XML annotation files and converted them to YOLO format labels in /content/datasets/labels


In [ ]:
import os
import shutil
from sklearn.model_selection import train_test_split

images_source_dir = '/content/datasets/images'
labels_source_dir = '/content/datasets/labels'
dataset_base_dir = '/content/datasets'

output_images_train_dir = os.path.join(dataset_base_dir, 'images', 'train')
output_images_val_dir = os.path.join(dataset_base_dir, 'images', 'val')
output_labels_train_dir = os.path.join(dataset_base_dir, 'labels', 'train')
output_labels_val_dir = os.path.join(dataset_base_dir, 'labels', 'val')

os.makedirs(output_images_train_dir, exist_ok=True)
os.makedirs(output_images_val_dir, exist_ok=True)
os.makedirs(output_labels_train_dir, exist_ok=True)
os.makedirs(output_labels_val_dir, exist_ok=True)

print(f"Created directories: {output_images_train_dir}, {output_images_val_dir}, {output_labels_train_dir}, {output_labels_val_dir}")

all_image_files = []
valid_extensions = ('.jpg', '.jpeg', '.png')
for fname in os.listdir(images_source_dir):
    if fname.lower().endswith(valid_extensions):
        all_image_files.append(fname)

print(f"Found {len(all_image_files)} image files.")

train_images, val_images = train_test_split(all_image_files, test_size=0.2, random_state=42)

print(f"Split into {len(train_images)} training images and {len(val_images)} validation images.")

for img_fname in train_images:
    src_image_path = os.path.join(images_source_dir, img_fname)
    dst_image_path = os.path.join(output_images_train_dir, img_fname)
    shutil.copyfile(src_image_path, dst_image_path)

    label_fname = img_fname.rsplit('.', 1)[0] + '.txt'
    src_label_path = os.path.join(labels_source_dir, label_fname)
    dst_label_path = os.path.join(output_labels_train_dir, label_fname)
    if os.path.exists(src_label_path):
        shutil.copyfile(src_label_path, dst_label_path)
    else:
        print(f"Warning: Label file {src_label_path} not found for image {img_fname}")

for img_fname in val_images:
    src_image_path = os.path.join(images_source_dir, img_fname)
    dst_image_path = os.path.join(output_images_val_dir, img_fname)
    shutil.copyfile(src_image_path, dst_image_path)

    label_fname = img_fname.rsplit('.', 1)[0] + '.txt'
    src_label_path = os.path.join(labels_source_dir, label_fname)
    dst_label_path = os.path.join(output_labels_val_dir, label_fname)
    if os.path.exists(src_label_path):
        shutil.copyfile(src_label_path, dst_label_path)
    else:
        print(f"Warning: Label file {src_label_path} not found for image {img_fname}")

print(f"Successfully moved {len(train_images)} images and labels to training set.")
print(f"Successfully moved {len(val_images)} images and labels to validation set.")

Created directories: /content/datasets/images/train, /content/datasets/images/val, /content/datasets/labels/train, /content/datasets/labels/val
Found 5000 image files.
Split into 4000 training images and 1000 validation images.
Successfully moved 4000 images and labels to training set.
Successfully moved 1000 images and labels to validation set.


In [ ]:
import yaml

dataset_base_dir = '/content/datasets'
train_images_path = 'images/train'
val_images_path = 'images/val'

data_yaml = {
    'path': dataset_base_dir,
    'train': train_images_path,
    'val': val_images_path,
    'nc': len(classes),
    'names': classes
}

data_yaml_path = os.path.join(dataset_base_dir, 'data.yaml')

with open(data_yaml_path, 'w') as file:
    yaml.dump(data_yaml, file, default_flow_style=False)

print(f"'data.yaml' created successfully at {data_yaml_path}:")
with open(data_yaml_path, 'r') as file:
    print(file.read())


'data.yaml' created successfully at /content/datasets/data.yaml:
names:
- helmet
- head
- person
nc: 3
path: /content/datasets
train: images/train
val: images/val



In [ ]:
from ultralytics import YOLO

model = YOLO('yolov8n.pt')

model.train(data=data_yaml_path, epochs=10, imgsz=640, project=SAVE_DIR)


Ultralytics 8.4.8 🚀 Python-3.12.12 torch-2.9.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, angle=1.0, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/datasets/data.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, end2end=None, epochs=10, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n.pt, momentum=0.937, mosaic=1.0, multi_scale=0.0, name=train, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=100, perspective=0.0, p

ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1, 2])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x789af3525b50>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0.04

In [ ]:
from ultralytics import YOLO
import os

best_model_path = os.path.join(SAVE_DIR, 'train', 'weights', 'best.pt')
model = YOLO(best_model_path)

metrics = model.val(data=data_yaml_path)

print("\n--- Model Evaluation Metrics ---")
print(f"mAP50-95: {metrics.box.map:.4f}")
print(f"mAP50: {metrics.box.map50:.4f}")
print(f"mAP75: {metrics.box.map75:.4f}")
print("Per-class metrics:")
for i, name in enumerate(metrics.names):
    print(f"  Class '{name}': P={metrics.box.p[i]:.4f}, R={metrics.box.r[i]:.4f}, mAP50={metrics.box.ap50[i]:.4f}, mAP50-95={metrics.box.ap[i]:.4f}")

Ultralytics 8.4.8 🚀 Python-3.12.12 torch-2.9.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
Model summary (fused): 73 layers, 3,006,233 parameters, 0 gradients, 8.1 GFLOPs
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 3403.3±934.3 MB/s, size: 251.8 KB)
val: Scanning /content/datasets/labels/val.cache... 1000 images, 0 backgrounds, 0 corrupt: 100% ━━━━━━━━━━━━ 1000/1000 262.1Mit/s 0.0s
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 63/63 3.8it/s 16.4s
                   all       1000       4981       0.94      0.576      0.621      0.409
                helmet        925       3783      0.954      0.882      0.954      0.626
                  head        177       1022      0.866      0.845      0.884      0.589
                person         39        176          1          0     0.0255     0.0117
Speed: 2.3ms preprocess, 4.0ms inference, 0.0ms loss, 2.0ms postprocess per image
Results saved to /content/runs/detect/val2

--- Model